# 1. Import Required Libraries
Import libraries such as pandas, numpy, and the LLM pipeline for data handling and cleaning.

In [ ]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

from llm_data_quality_assistant.enums import Models
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
import numpy as np
import jupyter_helper_functions
import string
import time

load_dotenv()

True

# 2. Load and Explore Flight Data
Load the flight dataset and perform exploratory data analysis to understand its structure and content.

In [ ]:
corrupt_dataset = jupyter_helper_functions.load_dataset(
    "../datasets/parker_datasets/flight/flightcleaned_corrupted_first1000.csv"
)
gold_standard = jupyter_helper_functions.load_dataset(
    "../datasets/parker_datasets/flight/flight_cleaned_gold_first1000.csv"
)

partial_keys = jupyter_helper_functions.load_text_file("../datasets/parker_datasets/flight/flight.partialkey")
rules = jupyter_helper_functions.load_text_file("../datasets/parker_datasets/flight/flight.rules")

# print(partial_keys)
# print(rules)
# print(corrupt_dataset.head(2))
# print(gold_standard.head(2))
# print(type(gold_standard.get("composed_key").iloc[0]))
# print(type(corrupt_dataset.get("composed_key").iloc[0]))

date_collected, flight_number ->
    scheduled_departure,
    actual_departure,
    scheduled_arrival,
    actual_arrival

-- Attributes
@scheduled_departure:datetime_in_minutes
@scheduled_arrival:datetime_in_minutes
@actual_departure:datetime_in_minutes
@actual_arrival:datetime_in_minutes

-- Sigma rules
scheduled_departure >= scheduled_arrival
actual_departure >= actual_arrival
actual_departure >= scheduled_departure
actual_departure <= scheduled_departure

                   composed_key  actual_arrival  actual_departure  \
0  2011-12-01 - AA-1007-MIA-PHX          3055.0            2756.0   
1  2011-12-01 - AA-1007-MIA-PHX          3055.0            2756.0   

   scheduled_arrival  scheduled_departure  
0             3065.0               2755.0  
1             3065.0               2755.0  
                   composed_key  actual_arrival  actual_departure  \
0  2011-12-01 - AA-1007-MIA-PHX          3055.0            2768.0   
1  2011-12-01 - AA-1007-MIA-PHX          3055.0           

# 3. Clean and Merge Data with LLM
Use the LLM pipeline to clean and merge the corrupted dataset using the provided rules and evaluate the results.

In [ ]:
from llm_data_quality_assistant.pipeline import Pipeline
from llm_data_quality_assistant.enums import Models
import jupyter_helper_functions
import string
import time
string.punctuation = string.punctuation.replace("'", "")  # Remove single quotes from punctuation

# Use a primary key for merging
primary_key = "composed_key"
model = Models.GeminiModels.GEMINI_2_0_FLASH_LITE
rows_of_context = 50

extra = "simple approach"
file_name = jupyter_helper_functions.sanitize_filename(f"{model.value}_{rows_of_context}_rows_context_{extra}")   

rpm = 30
additional_prompt = f"""
Here are rows of the dataset to provide context for the cleaning process:
{corrupt_dataset.sample(rows_of_context).to_string(index=False)}
"""

# Merge/clean with LLM
merged_df, time_taken = jupyter_helper_functions.merge_with_llm_timed(
    dataset = corrupt_dataset,
    primary_key = primary_key,
    model = model,
    rpm = rpm,
    additional_prompt = additional_prompt
    )


Merging groups with LLM: 100%|██████████| 1000/1000 [33:32<00:00,  2.01s/it]

                       composed_key  actual_arrival  actual_departure  \
0      2011-12-01 - AA-1007-MIA-PHX          3055.0            2769.0   
1      2011-12-01 - AA-1007-MIA-PHX          3055.0            2769.0   
2      2011-12-01 - AA-1007-MIA-PHX          3055.0            2769.0   
3      2011-12-01 - AA-1007-MIA-PHX          3055.0            2769.0   
4      2011-12-01 - AA-1007-MIA-PHX          3055.0            2769.0   
...                             ...             ...               ...   
24601   2012-01-03 - UA-938-DEN-ORD         50471.0           50359.0   
24602   2012-01-03 - UA-938-DEN-ORD         50471.0           50359.0   
24603   2012-01-03 - UA-938-DEN-ORD         50471.0           50359.0   
24604   2012-01-03 - UA-938-DEN-ORD         50471.0           50359.0   
24605   2012-01-03 - UA-938-DEN-ORD         50471.0           50359.0   

       scheduled_arrival  scheduled_departure  
0                 3065.0               2755.0  
1                 3065.0   

# 4. Evaluate the Results
Evaluate the cleaned dataset using micro and macro evaluation metrics.

In [ ]:
# (Evaluation is now handled by standardize_and_evaluate)

MICRO EVALUATION RESULTS
{'accuracy': 0.7491018450784361,
 'column_names': ['composed_key',
                  'actual_arrival',
                  'actual_departure',
                  'scheduled_arrival',
                  'scheduled_departure'],
 'f1_score': 0.6398553260996384,
 'false_negative': 20042,
 'false_negative_rate': 0.4222657649116154,
 'false_positive': 10826,
 'false_positive_rate': 0.14326359389680682,
 'num_columns': 5,
 'num_rows': 24606,
 'precision': 0.7169451198786833,
 'recall': 0.5777342350883846,
 'true_negative': 64741,
 'true_positive': 27421}
MACRO EVALUATION RESULTS
{'column_names': ['composed_key',
                  'actual_arrival',
                  'actual_departure',
                  'scheduled_arrival',
                  'scheduled_departure'],
 'num_columns': 5,
 'num_rows': 24606,
 'stats': [{'accuracy': 1.0,
            'column_name': 'composed_key',
            'f1_score': 0.0,
            'false_negative': 0,
            'false_negative_rate': 0.0

In [ ]:
import json
import time
# Save merged dataset
jupyter_helper_functions.save_dataframe_csv(merged_df, f"../analysis/repairs/flight/merged_dataset_{file_name}.csv")

# Evaluate results
jupyter_helper_functions.standardize_and_evaluate(
    gold_standard=gold_standard,
    merged_df=merged_df,
    corrupt_dataset=corrupt_dataset,
    primary_key=primary_key,
    time_delta=time_taken,
    results_dir=f"../analysis/results/flight/",
    file_name=file_name,
)
